# Improving Access to Clean, Safe Drinking Water in Tanzania
## Exploratory Data Analysis - Part 2

Kenneth Liao

---

## Background

The UN publishes and reviews a list of least developed countries (LDC) every 3 years. LDCs are “low-income countries confronting severe structural impediments to sustainable development. They are highly vulnerable to economic and environmental shocks and have low levels of human assets.”$^{1}$. Tanzania has been classified as an LDC since the UN published the first list of LDCs in 1971$^{2}$. A common challenge of LDCs is a lack of infrastructure to support the development of the nation, including access to education and healthcare, waste management, and potable water.

According to UNICEF, as of 2017, more than 24 million Tanzanians lacked access to basic drinking water$^{3}$. This corresponds to only 56.7% of the country’s population having access to basic drinking water. Outside of developed urban areas, much of the potable water is accessed via water pumps. 

Taarifa is an open-source platform for crowd-sourced reporting and triaging of infrastructure related issues. Together with the Tanzanian Ministry of Water, data has been collected for thousands of water pumps throughout Tanzania. The goal of this project is to be able to predict the condition of these water pumps to improve maintenance, reduce pump downtime, and ensure basic water access for tens of millions of Tanzanians.

**References**

1. https://www.un.org/development/desa/dpad/least-developed-country-category.html
2. https://www.un.org/development/desa/dpad/wp-content/uploads/sites/45/publication/ldc_list.pdf
3. https://washwatch.org/en/countries/tanzania/summary/statistics/


### Problem Description

Predict the operating condition of water pumps in Tanzania given various metadata on each water pump.

### Strategy

The strategy will be to implement a Random Forest model for multiclass classification of the state of water pumps.

### Data

The dataset is provided by Taarifa, together with the Tanzanian Ministry of Water and is hosted by DrivenData.org:

https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/23/

---

## Exploratory Data Analysis

Start by importing the necessary libraries and loading the data.

In [1]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.express as px
from config import credentials

init_notebook_mode(connected=True)

In [3]:
# load the data
train = pd.read_csv('data/train.csv')
train_labels = pd.read_csv('data/train-labels.csv')

`amount_tsh` - Total static head (amount water available to waterpoint)
<br>`date_recorded` - The date the row was entered
<br>`funder` - Who funded the well
<br>`gps_height` - Altitude of the well
<br>`installer` - Organization that installed the well
<br>`longitude` - GPS coordinate
<br>`latitude` - GPS coordinate
<br>`wpt_name` - Name of the waterpoint if there is one
<br>`num_private` -
<br>`basin` - Geographic water basin
<br>`subvillage` - Geographic location
<br>`region` - Geographic location
<br>`region_code` - Geographic location (coded)
<br>`district_code` - Geographic location (coded)
<br>`lga` - Geographic location
<br>`ward` - Geographic location
<br>`population` - Population around the well
<br>`public_meeting` - True/False
<br>`recorded_by` - Group entering this row of data
<br>`scheme_management` - Who operates the waterpoint
<br>`scheme_name` - Who operates the waterpoint
<br>`permit` - If the waterpoint is permitted
<br>`construction_year` - Year the waterpoint was constructed
<br>`extraction_type` - The kind of extraction the waterpoint uses
<br>`extraction_type_group` - The kind of extraction the waterpoint uses
<br>`extraction_type_class` - The kind of extraction the waterpoint uses
<br>`management` - How the waterpoint is managed
<br>`management_group` - How the waterpoint is managed
<br>`payment` - What the water costs
<br>`payment_type` - What the water costs
<br>`water_quality` - The quality of the water
<br>`quality_group` - The quality of the water
<br>`quantity` - The quantity of water
<br>`quantity_group` - The quantity of water
<br>`source` - The source of the water
<br>`source_type` - The source of the water
<br>`source_class` - The source of the water
<br>`waterpoint_type` - The kind of waterpoint
<br>`waterpoint_type_group` - The kind of waterpoint

In [4]:
def plot_hist(data, col, ylog=False, xlog=False):
    """
    Convenience function for plotting the histogram
    for a given feature.
    """
    
    if ylog:
        ymode='log'
    else:
        ymode=None
    if xlog:
        xmode='log'
    else:
        xmode=None
    
    trace = go.Histogram(x=data[col], name='col')
    
    layout = go.Layout(title=f'{col} Distribution',
                  yaxis=dict(title='Count', type=ymode),
                       xaxis=dict(type=xmode))
        
    fig = go.Figure([trace], layout=layout)
    
    iplot(fig, filename=f'{col}-dist.html')

### What is the most common type of pump?

In [13]:
train.set_index('id', drop=True, inplace=True)
train_labels.set_index('id', drop=True, inplace=True)
full_data = pd.concat([train, train_labels], axis=1)
full_data.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [30]:
status_by_pump = full_data[['extraction_type', 'date_recorded', 'status_group']].groupby(['extraction_type', 'status_group']).count()
status_by_pump = status_by_pump.unstack(-1).droplevel(0, axis=1)
status_by_pump

status_group,functional,functional needs repair,non functional
extraction_type,,,
afridev,1200.0,42.0,528.0
cemo,45.0,9.0,36.0
climax,8.0,NaN,24.0
gravity,16048.0,2701.0,8031.0
india mark ii,1448.0,79.0,873.0
india mark iii,44.0,1.0,53.0
ksb,703.0,26.0,686.0
mono,1082.0,129.0,1654.0
nira/tanira,5421.0,641.0,2092.0


In [41]:
#status_by_pump_proportion = 
status_by_pump_proportion = status_by_pump.divide(status_by_pump.sum(axis=1), axis=0)
status_by_pump_proportion

status_group,functional,functional needs repair,non functional
extraction_type,,,
afridev,0.677966,0.023729,0.298305
cemo,0.500000,0.100000,0.400000
climax,0.250000,NaN,0.750000
gravity,0.599253,0.100859,0.299888
india mark ii,0.603333,0.032917,0.363750
india mark iii,0.448980,0.010204,0.540816
ksb,0.496820,0.018375,0.484806
mono,0.377661,0.045026,0.577312
nira/tanira,0.664827,0.078612,0.256561
